Load the data.

In [1]:
import pandas as pd
url="https://bd29ee0e-54ab-4daa-9671-d153865d1620.usrfiles.com/ugd/bd29ee_6890ec66fb17468c94c91a9740379182.csv"
df=pd.read_csv(url,header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,1.14,3.0,290.0,1350.0,6680.0,7.58,7.58,464.0,288.0,551.0,1.10,0.998,5.95,1.02,7.14,0.082,0.95,0.975
1,2.09,6.0,6960.0,1380.0,6830.0,28.20,28.20,635.0,288.0,582.0,1.33,0.998,7.28,1.02,10.70,0.287,0.95,0.975
2,3.14,9.0,8380.0,1390.0,7110.0,60.40,60.40,606.0,288.0,588.0,1.39,0.998,7.57,1.02,13.10,0.259,0.95,0.975
3,4.16,12.0,14700.0,1550.0,7790.0,114.00,114.00,661.0,288.0,614.0,1.66,0.998,9.01,1.02,18.10,0.358,0.95,0.975
4,5.14,15.0,21600.0,1920.0,8490.0,175.00,175.00,731.0,288.0,646.0,2.08,0.998,11.20,1.03,26.40,0.522,0.95,0.975


In [2]:
df.shape

(11934, 18)

According to the dataset description at UCI's website, the dataset has no missing values. 

In [3]:
df.isna().any()

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
dtype: bool

Separate the predictor variables and the target variables.

In [4]:
X=df.iloc[:,0:16]
X.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,1.14,3.0,290.0,1350.0,6680.0,7.58,7.58,464.0,288.0,551.0,1.10,0.998,5.95,1.02,7.14,0.082
1,2.09,6.0,6960.0,1380.0,6830.0,28.20,28.20,635.0,288.0,582.0,1.33,0.998,7.28,1.02,10.70,0.287
2,3.14,9.0,8380.0,1390.0,7110.0,60.40,60.40,606.0,288.0,588.0,1.39,0.998,7.57,1.02,13.10,0.259
3,4.16,12.0,14700.0,1550.0,7790.0,114.00,114.00,661.0,288.0,614.0,1.66,0.998,9.01,1.02,18.10,0.358
4,5.14,15.0,21600.0,1920.0,8490.0,175.00,175.00,731.0,288.0,646.0,2.08,0.998,11.20,1.03,26.40,0.522


In [5]:
y16=pd.DataFrame(df.iloc[:,16]).rename(columns={16:"Compressor_coef"})
y16.head()

,Compressor_coef
0,0.95
1,0.95
2,0.95
3,0.95
4,0.95


In [6]:
y17=pd.DataFrame(df.iloc[:,17]).rename(columns={17:"Turbine_coef"})
y17.head()

,Turbine_coef
0,0.975
1,0.975
2,0.975
3,0.975
4,0.975


Feature selection.

In [7]:
import numpy as np
from numpy import set_printoptions
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_regression

# feature extraction. We are going to output the selection scores for all features and select the features with the highest scores.
test = SelectKBest(score_func=mutual_info_regression, k="all")
fit = test.fit(X, np.asarray(y16).reshape(y16.shape[0],))
features = fit.transform(X)
# summarize scores
set_printoptions(precision=3)

scores=fit.scores_
scores=pd.DataFrame(scores)
scores=scores.rename(columns={0:"scores"})
scores=scores.sort_values(by=["scores"],ascending=False)
scores

,scores
9,0.836038
4,0.641331
10,0.386150
7,0.211988
15,0.108184
5,0.086627
6,0.085767
14,0.084815
2,0.017756
3,0.014055


In [8]:
scores16=scores[scores["scores"]>0]
scores16

,scores
9,0.836038
4,0.641331
10,0.386150
7,0.211988
15,0.108184
5,0.086627
6,0.085767
14,0.084815
2,0.017756
3,0.014055


In [9]:
X16=X.iloc[:,scores16.index]
X16.head()

,9,4,10,7,15,5,6,14,2,3,11
0,551.0,6680.0,1.10,464.0,0.082,7.58,7.58,7.14,290.0,1350.0,0.998
1,582.0,6830.0,1.33,635.0,0.287,28.20,28.20,10.70,6960.0,1380.0,0.998
2,588.0,7110.0,1.39,606.0,0.259,60.40,60.40,13.10,8380.0,1390.0,0.998
3,614.0,7790.0,1.66,661.0,0.358,114.00,114.00,18.10,14700.0,1550.0,0.998
4,646.0,8490.0,2.08,731.0,0.522,175.00,175.00,26.40,21600.0,1920.0,0.998


Based on the result of the feature selection algorithm, for the first target variable, GT Compressor decay state coefficient, I am going to use only 11 of the 16 predictor variables.

In [10]:
fit = test.fit(X, np.asarray(y17).reshape(y17.shape[0],))
features = fit.transform(X)
# summarize scores
set_printoptions(precision=3)

scores=fit.scores_
scores=pd.DataFrame(scores)
scores=scores.rename(columns={0:"scores"})
scores=scores.sort_values(by=["scores"],ascending=False)
scores

,scores
12,0.907416
14,0.220307
15,0.206124
4,0.118192
10,0.062529
7,0.058355
13,0.004266
11,0.002262
0,0.000000
1,0.000000


In [11]:
scores17=scores[scores["scores"]>0]
scores17

,scores
12,0.907416
14,0.220307
15,0.206124
4,0.118192
10,0.062529
7,0.058355
13,0.004266
11,0.002262


In [12]:
X17=X.iloc[:,scores17.index]
X17.head()

,12,14,15,4,10,7,13,11
0,5.95,7.14,0.082,6680.0,1.10,464.0,1.02,0.998
1,7.28,10.70,0.287,6830.0,1.33,635.0,1.02,0.998
2,7.57,13.10,0.259,7110.0,1.39,606.0,1.02,0.998
3,9.01,18.10,0.358,7790.0,1.66,661.0,1.02,0.998
4,11.20,26.40,0.522,8490.0,2.08,731.0,1.03,0.998


Based on the result of feature selection algorithem, for the second target variable, GT Turbine decay state coefficient, I am only going to use 8 out of the 16 predictor variables.

Regression model for the GT Compressor Decay Coefficient.

In [13]:
from sklearn.model_selection import train_test_split
X16_train, X16_test, y16_train, y16_test = train_test_split(X16, y16, test_size=0.2, random_state=42)
print(X16_train.shape)
print(X16_test.shape)

(9547, 11)
(2387, 11)


In [14]:
from sklearn.ensemble import GradientBoostingRegressor
model=GradientBoostingRegressor(loss='ls',learning_rate=0.001,n_estimators=3000,max_features='sqrt',criterion="mse",max_depth=45,random_state=0).fit(X16_train,y16_train)
model

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingRegressor(alpha=0.9, criterion='mse', init=None,
             learning_rate=0.001, loss='ls', max_depth=45,
             max_features='sqrt', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=3000,
             n_iter_no_change=None, presort='auto', random_state=0,
             subsample=1.0, tol=0.0001, validation_fraction=0.1, verbose=0,
             warm_start=False)

In [15]:
from sklearn.metrics import r2_score
y16hat=model.predict(X16_train)
print("R-squared for the GT compressor coefficient on train data is: ",r2_score(y16_train,y16hat))

R-squared for the GT compressor coefficient on train data is:  0.9957539096454002


In [16]:
y16hat=model.predict(X16_test)
print("R-squared for the GT compressor coefficient on train data is: ",r2_score(y16_test,y16hat))

R-squared for the GT compressor coefficient on train data is:  0.9861067799499837


In [17]:
y16hat=model.predict(X16_train)
y16hat=np.asarray(y16hat).reshape(y16hat.shape[0],)
y16_train=np.asarray(y16_train).reshape(y16_train.shape[0],)
print("Absolute error for the GT compressor coefficient on train data is: \n",abs(y16_train-y16hat))

Absolute error for the GT compressor coefficient on train data is: 
 [5.267e-04 2.021e-04 4.037e-05 ... 3.828e-04 1.155e-03 7.728e-04]


In [18]:
y16hat=model.predict(X16_test)
y16hat=np.asarray(y16hat).reshape(y16hat.shape[0],)
y16_test=np.asarray(y16_test).reshape(y16_test.shape[0],)
print("Absolute error for the GT compressor coefficient on test data is: \n",abs(y16_test-y16hat))

Absolute error for the GT compressor coefficient on test data is: 
 [0.003 0.    0.001 ... 0.002 0.003 0.001]


In [19]:
AE=pd.DataFrame(abs(y16_test-y16hat)).rename(columns={0:"AbsoluteError_TestData"}).sort_values(by=["AbsoluteError_TestData"],ascending=False)
AE

,AbsoluteError_TestData
1690,9.594257e-03
126,7.667058e-03
2362,7.585238e-03
249,7.040776e-03
153,7.019541e-03
...,...
1749,1.821537e-06
1926,1.211592e-06
958,9.559897e-07
67,8.721025e-07


In [20]:
from sklearn.metrics import mean_squared_error as mse
print("The root mean squared error on test data is: ",np.sqrt(mse(y16_test,y16hat)))

The root mean squared error on test data is:  0.0017281945107356356


Regression model for the GT Turbine Coefficient.

In [14]:
from sklearn.model_selection import train_test_split
X17_train, X17_test, y17_train, y17_test = train_test_split(X17, y17, test_size=0.2, random_state=42)
print(X17_train.shape)
print(X17_test.shape)

(9547, 8)
(2387, 8)


In [17]:
from sklearn.ensemble import GradientBoostingRegressor
model17=GradientBoostingRegressor(loss='ls',learning_rate=0.001,n_estimators=3000,max_features='sqrt',criterion="mse",max_depth=45,random_state=0).fit(X17_train,y17_train)
model17

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingRegressor(alpha=0.9, criterion='mse', init=None,
             learning_rate=0.001, loss='ls', max_depth=45,
             max_features='sqrt', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=3000,
             n_iter_no_change=None, presort='auto', random_state=0,
             subsample=1.0, tol=0.0001, validation_fraction=0.1, verbose=0,
             warm_start=False)

In [21]:
from sklearn.metrics import r2_score
y17hat=model17.predict(X17_train)
print("R-squared for the GT Turbine coefficient on train data is: ",r2_score(y17_train,y17hat))

R-squared for the GT Turbine coefficient on train data is:  0.994932837120402


In [23]:
y17hat=model17.predict(X17_test)
print("R-squared for the GT compressor coefficient on train data is: ",r2_score(y17_test,y17hat))

R-squared for the GT compressor coefficient on train data is:  0.9734362162951687


In [25]:
y17hat=model17.predict(X17_train)
y17hat=np.asarray(y17hat).reshape(y17hat.shape[0],)
y17_train=np.asarray(y17_train).reshape(y17_train.shape[0],)
print("Absolute error for the GT Turbine coefficient on train data is: \n",abs(y17_train-y17hat))

Absolute error for the GT Turbine coefficient on train data is: 
 [0.    0.    0.001 ... 0.001 0.    0.001]


In [26]:
y17hat=model17.predict(X17_test)
y17hat=np.asarray(y17hat).reshape(y17hat.shape[0],)
y17_test=np.asarray(y17_test).reshape(y17_test.shape[0],)
print("Absolute error for the GT Turbine coefficient on test data is: \n",abs(y17_test-y17hat))

Absolute error for the GT Turbine coefficient on test data is: 
 [8.702e-05 1.948e-03 4.546e-04 ... 6.008e-05 9.015e-04 9.870e-04]


In [27]:
AE17=pd.DataFrame(abs(y17_test-y17hat)).rename(columns={0:"AbsoluteError_TestData"}).sort_values(by=["AbsoluteError_TestData"],ascending=False)
AE17

,AbsoluteError_TestData
1845,1.549965e-02
1322,1.028307e-02
1555,1.014463e-02
2231,9.685348e-03
1467,9.065795e-03
...,...
436,9.061503e-07
917,7.804063e-07
2261,7.438651e-07
687,4.820818e-07


In [28]:
from sklearn.metrics import mean_squared_error as mse
print("The root mean squared error on test data is: ",np.sqrt(mse(y17_test,y17hat)))

The root mean squared error on test data is:  0.0012303263876443395


Multiple output regression model with no feature selection.

In [5]:
y=df.iloc[:,[16,17]]
y.head()

,16,17
0,0.95,0.975
1,0.95,0.975
2,0.95,0.975
3,0.95,0.975
4,0.95,0.975


In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape)
print(X_test.shape)

(9547, 16)
(2387, 16)


In [7]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import GradientBoostingRegressor

In [9]:
multi_model=MultiOutputRegressor(GradientBoostingRegressor(loss='ls',learning_rate=0.001,n_estimators=3000,max_features='sqrt',criterion="mse",max_depth=45,random_state=0)).fit(X_train,y_train)
multi_model

MultiOutputRegressor(estimator=GradientBoostingRegressor(alpha=0.9, criterion='mse', init=None,
             learning_rate=0.001, loss='ls', max_depth=45,
             max_features='sqrt', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_s...       subsample=1.0, tol=0.0001, validation_fraction=0.1, verbose=0,
             warm_start=False),
           n_jobs=None)

In [12]:
from sklearn.metrics import mean_squared_error as mse
import numpy as np
y_pred=pd.DataFrame(multi_model.predict(X_test))
n_classes=y_test.shape[1]
for i in range(n_classes):
    print("The root mean square error on test data for the ",i,"th target is ,", np.sqrt(mse(y_test.iloc[:,i],y_pred.iloc[:,i])))

The root mean square error on test data for the  0 th target is , 0.0016300542694567199
The root mean square error on test data for the  1 th target is , 0.0010498297906512818


In [13]:
from sklearn.metrics import r2_score as r2
for i in range(n_classes):
    print("The r-squared on test data for the ",i,"th target is ,", r2(y_test.iloc[:,i],y_pred.iloc[:,i], multioutput='variance_weighted'))

The r-squared on test data for the  0 th target is , 0.987639905278979
The r-squared on test data for the  1 th target is , 0.9806586384742164


In [36]:
AE=pd.DataFrame(abs(np.asarray(y_test)-np.asarray(y_pred))).rename(columns={0:"AE16",1:"AE17"}).sort_values(by=["AE17"],ascending=False)
AE

,AE16,AE17
1845,0.005788,0.014048
1467,0.002461,0.007094
1766,0.004631,0.006793
1783,0.005779,0.006315
742,0.004213,0.006196
...,...,...
107,0.001143,0.000002
918,0.000716,0.000002
1597,0.001755,0.000001
415,0.001979,0.000001


With the same hyper-parameters, even without feature selection, the multiple output regression mode by piling MultiOutputRegressor on top of GradientBoostingRegressor performs slightly better than the individual models, especially on the second target variable.